This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](kochanek-bartels.ipynb)

# Uniform Kochanek--Bartels Splines

As a starting point,
remember the tangent vectors from
[Catmull-Rom splines](catmull-rom.ipynb):

\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{
(\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{equation*}

## Parameters

*TCB splines* are all about inserting the parameters
$T$, $C$ and $B$ into this equation.

### Tension

see equation 4 in
<cite data-cite="kochanek1984tcb">the Kochanek/Bartels (1984) paper</cite>

\begin{equation*}
\boldsymbol{\dot{x}}_i = (1 - T_i) \frac{
(\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{equation*}

### Continuity

Up to now,
the goal was to have a continuous first derivative at the control points,
i.e. the incoming and outgoing tangent vectors were identical:

\begin{equation*}
\boldsymbol{\dot{x}}_i = \boldsymbol{\dot{x}}_i^{(-)} = \boldsymbol{\dot{x}}_i^{(+)}
\end{equation*}

This also happens to be the requirement for a spline to be $C^1$ continuous.

The "continuity" parameter allows us to break this continuity if we so desire,
leading to different incoming and outgoing tangent vectors
(see equations 5 and 6 in
<cite data-cite="kochanek1984tcb">the Kochanek/Bartels (1984) paper</cite>):

\begin{align*}
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
(1 - C_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 + C_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}\\
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
(1 + C_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - C_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{align*}

### Bias

see equation 7 in
<cite data-cite="kochanek1984tcb">the Kochanek/Bartels (1984) paper</cite>


\begin{equation*}
\boldsymbol{\dot{x}}_i = \frac{
(1 + B_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - B_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{equation*}

### All Three Combined

see equations 8 and 9 in
<cite data-cite="kochanek1984tcb">the Kochanek/Bartels (1984) paper</cite>

\begin{align*}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
(1 - T_i) (1 + C_i) (1 + B_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - T_i) (1 - C_i) (1 - B_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
(1 - T_i) (1 - C_i) (1 + B_i) (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(1 - T_i) (1 + C_i) (1 - B_i) (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{align*}

Note: There is an error in equation (6.11) of 
<cite data-cite="millington2009matrices">Ian Millington's paper</cite>
(all subscripts of $x$ are wrong, most likely copy-pasted from the preceding equation).

To simplify the result we will get later, we introduce the following shorthands
(as suggested in
<cite data-cite="millington2009matrices">Millington's paper</cite>):

\begin{align*}
a_i &= (1 - T_i) (1 + C_i) (1 + B_i)\\
b_i &= (1 - T_i) (1 - C_i) (1 - B_i)\\
c_i &= (1 - T_i) (1 - C_i) (1 + B_i)\\
d_i &= (1 - T_i) (1 + C_i) (1 - B_i)
\end{align*}

This leads to the simplified equations

\begin{align*}
\boldsymbol{\dot{x}}_i^{(+)} &= \frac{
a_i (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
b_i (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}\\
\boldsymbol{\dot{x}}_i^{(-)} &= \frac{
c_i (\boldsymbol{x}_i - \boldsymbol{x}_{i-i}) +
d_i (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{2}
\end{align*}

## Calculation

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
from utility import NamedExpression, NamedMatrix

[helper.py](helper.py)

In [ ]:
from helper import plot_basis

Same control values as Catmull-Rom ...

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
control_values_KB = sp.Matrix([x3, x4, x5, x6])
control_values_KB

... but three additional parameters per vertex.
In our calculation, the parameters belonging to $\boldsymbol{x}_4$ and $\boldsymbol{x}_5$ are relevant:

In [ ]:
T4, T5 = sp.symbols('T4 T5')
C4, C5 = sp.symbols('C4 C5')
B4, B5 = sp.symbols('B4 B5')

In [ ]:
a4 = NamedExpression('a4', (1 - T4) * (1 + C4) * (1 + B4))
b4 = NamedExpression('b4', (1 - T4) * (1 - C4) * (1 - B4))
c5 = NamedExpression('c5', (1 - T5) * (1 - C5) * (1 + B5))
d5 = NamedExpression('d5', (1 - T5) * (1 + C5) * (1 - B5))
display(a4, b4, c5, d5)

In [ ]:
xd4 = NamedExpression(
    'xdotbm4^(+)',
    sp.S.Half * (a4.name * (x4 - x3) + b4.name * (x5 - x4)))
xd5 = NamedExpression(
    'xdotbm5^(-)',
    sp.S.Half * (c5.name * (x5 - x4)  + d5.name * (x6 - x5)))
display(xd4, xd5)

In [ ]:
display(xd4.subs_symbols(a4, b4))
display(xd5.subs_symbols(c5, d5))

Same as with Catmull-Rom, try to find a transformation from cardinal control values to Hermite control values.
This can be used to get the full basis matrix.

In [ ]:
control_values_H = sp.Matrix([x4, x5, xd4.name, xd5.name])
control_values_H

From the [notebook about uniform Hermite splines](hermite-uniform.ipynb):

In [ ]:
M_H = NamedMatrix(
    r'{M_\text{H}}',
    sp.Matrix([[2, -2, 1, 1],
               [-3, 3, -2, -1],
               [0, 0, 1, 0],
               [1, 0, 0, 0]]))
M_H

In [ ]:
M_KBtoH = NamedMatrix(r'{M_{\text{KB$,4\to$H}}}', 4, 4)
M_KB = NamedMatrix(r'{M_{\text{KB},4}}', M_H.name * M_KBtoH.name)
M_KB

In [ ]:
NamedMatrix(control_values_H, M_KBtoH.name * control_values_KB)

If we substitute the above definitions of $\boldsymbol{\dot{x}}_4$ and $\boldsymbol{\dot{x}}_5$, we can directly read off the matrix elements:

In [ ]:
M_KBtoH.expr = sp.Matrix([
    [expr.coeff(cv) for cv in control_values_KB]
    for expr in control_values_H.subs([xd4.args, xd5.args]).expand()])
M_KBtoH.pull_out(sp.S.Half)

In [ ]:
M_KB = M_KB.subs_symbols(M_H, M_KBtoH).doit()
M_KB.pull_out(sp.S.Half)

And for completeness' sake, its inverse:

In [ ]:
M_KB.I

In [ ]:
t = sp.symbols('t')

In [ ]:
b_KB = NamedMatrix(r'{b_{\text{KB},4}}', sp.Matrix([t**3, t**2, t, 1]).T * M_KB.expr)
b_KB.T.pull_out(sp.S.Half)

To be able to plot the basis functions,
let's substitute $a_4$, $b_4$, $c_5$ and $d_5$ back in (which isn't pretty):

In [ ]:
b_KB = b_KB.subs_symbols(a4, b4, c5, d5).simplify()
b_KB.T.pull_out(sp.S.Half)

In [ ]:
labels = sp.symbols('xbm_i-1 xbm_i xbm_i+1 xbm_i+2')

In [ ]:
plot_basis(
    *b_KB.expr.subs({T4: 0, T5: 0, C4: 0, C5: 1, B4: 0, B5: 0}),
    labels=labels)

In [ ]:
plot_basis(
    *b_KB.expr.subs({T4: 0, T5: 0, C4: 0, C5: -0.5, B4: 0, B5: 0}),
    labels=labels)

TODO: plot some example curves